# Table of Contents
 <p><div class="lev1 toc-item"><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></div><div class="lev2 toc-item"><a href="#Generating-base-FASTA-with-intensities" data-toc-modified-id="Generating-base-FASTA-with-intensities-11"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Generating base-FASTA with intensities</a></div><div class="lev2 toc-item"><a href="#Simulating-and-quantifying-the-data" data-toc-modified-id="Simulating-and-quantifying-the-data-12"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Simulating and quantifying the data</a></div><div class="lev2 toc-item"><a href="#Generating-intensity-matrix" data-toc-modified-id="Generating-intensity-matrix-13"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Generating intensity matrix</a></div><div class="lev2 toc-item"><a href="#Dependencies" data-toc-modified-id="Dependencies-14"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Dependencies</a></div><div class="lev1 toc-item"><a href="#Prepare-for-simulation" data-toc-modified-id="Prepare-for-simulation-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Prepare for simulation</a></div><div class="lev2 toc-item"><a href="#Setup" data-toc-modified-id="Setup-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Setup</a></div><div class="lev2 toc-item"><a href="#Prepare-design-matrix" data-toc-modified-id="Prepare-design-matrix-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Prepare design matrix</a></div><div class="lev3 toc-item"><a href="#Setup-variables-with-sample-names" data-toc-modified-id="Setup-variables-with-sample-names-221"><span class="toc-item-num">2.2.1&nbsp;&nbsp;</span>Setup variables with sample names</a></div><div class="lev2 toc-item"><a href="#Generate-intensity-annotated-FASTA" data-toc-modified-id="Generate-intensity-annotated-FASTA-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Generate intensity-annotated FASTA</a></div><div class="lev1 toc-item"><a href="#Simulate-MS-data" data-toc-modified-id="Simulate-MS-data-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Simulate MS-data</a></div><div class="lev1 toc-item"><a href="#Quantification" data-toc-modified-id="Quantification-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Quantification</a></div><div class="lev2 toc-item"><a href="#Prepare-quantity-matrix" data-toc-modified-id="Prepare-quantity-matrix-41"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Prepare quantity matrix</a></div>

# Introduction

## Generating base-FASTA with intensities

The purpose with this notebook is to generate a simulated proteomics label-free quantification dataset. This is done by first generating a FASTA-file with headers in the following format:

```
>sp|P75925|C56I_ECOLI [# intensity=974548.6052639355 #]
MSFTNTPERYGVISAAFHWLSAIIVYGMFALGLWMVTLSYYDGWYHKAPELHKSIGILLM
```

The FASTA is generated from two separate FASTA files where entries in the first is used as background while entries from the second is used as spike-in. The script `lfqtk.py generat_spikein_set` varies intensities on features on the following conditions:

* A base-line intensity is set (on protein level)
* For the spike-in data intensities are set based on the base intensity times a sample-specific fold change
* The batch effect is applied as a constant intensity shift for all proteins in particular samples
* A random noise factor is applied

As this is performed on protein level intensities will be varying between peptides.

## Simulating and quantifying the data

Simulation of the data is performed using the OpenMS tool `MSSimulator`. The simulation is performed using default settings and output is received in `featureXML` format. The features in this files are linked to the identities of the entries used to generate the features.

The quantification is performed using the OpenMS tool `ProteinQuantifier`, which is ran with default settings. Quantification is only performed on peptide level.

## Generating intensity matrix

The final step is to merge the resulting tab-delimited files from the quantification step to a matrix representing the intensities for each peptide/sample. This is performed with a custom script `combine_quant_pd.py` which simply links intensities from identical peptide + protein identification. This is reasonable here as we know that the identifications are correct and unique from the simulated data.

## Dependencies

To run this notebook, the following dependencies are required. For demonstration purposes, resulting data files are included in the repository meaning that the analysis itself can be tested independently without running through this notebook.

* OpenMS binaries in PATH (version 2.2.0)
* Jupyter Bash kernel (https://github.com/takluyver/bash_kernel)
* GNU parallel (https://www.gnu.org/software/parallel/)

# Prepare for simulation

## Setup

In [1]:
threads=7
run=batch_nobatch
mkdir ${run}

## Prepare design matrix

This tab separated file contains information about each sample that will be generated - Which spike-in condition it belongs to and in which batch the sample was processed.

This design matrix is also used in the analysis step together with the resulting intensity matrix.

In [2]:
echo -e "name\tbiorepgroup\ttechrepgroup\tcondition\tbatch" > ${run}/design.tsv
echo -e "a1\t1\t1\ta\t1" >> ${run}/design.tsv
echo -e "a2\t2\t1\ta\t1" >> ${run}/design.tsv
echo -e "a3\t3\t1\ta\t1" >> ${run}/design.tsv
echo -e "a4\t4\t1\ta\t2" >> ${run}/design.tsv
echo -e "a5\t5\t1\ta\t2" >> ${run}/design.tsv
echo -e "a6\t6\t1\ta\t2" >> ${run}/design.tsv
echo -e "b1\t1\t1\tb\t1" >> ${run}/design.tsv
echo -e "b2\t2\t1\tb\t1" >> ${run}/design.tsv
echo -e "b3\t3\t1\tb\t1" >> ${run}/design.tsv
echo -e "b4\t4\t1\tb\t2" >> ${run}/design.tsv
echo -e "b5\t5\t1\tb\t2" >> ${run}/design.tsv
echo -e "b6\t6\t1\tb\t2" >> ${run}/design.tsv

In [3]:
cat ${run}/design.tsv

name	biorepgroup	techrepgroup	condition	batch
a1	1	1	a	1
a2	2	1	a	1
a3	3	1	a	1
a4	4	1	a	2
a5	5	1	a	2
a6	6	1	a	2
b1	1	1	b	1
b2	2	1	b	1
b3	3	1	b	1
b4	4	1	b	2
b5	5	1	b	2
b6	6	1	b	2


### Setup variables with sample names

A string with the sample names and a string with comma-delimited filenames are generated.

In [4]:
sample_names=$(cut -f1 ${run}/design.tsv \
    | tr "\n" " " | cut -f2- -d" ")
file_names=$(cut -f1 ${run}/design.tsv \
    | tr "\n" " " | cut -f2- -d" " \
    | sed "s/ /.fa,/g" | sed "s/,$//")
batch=$(cut -f2 ${run}/design.tsv | tr "\n" " " \
    | cut -f2- -d" ")

echo ${sample_names}
echo ${file_names}
echo ${batch}

a1 a2 a3 a4 a5 a6 b1 b2 b3 b4 b5 b6
a1.fa,a2.fa,a3.fa,a4.fa,a5.fa,a6.fa,b1.fa,b2.fa,b3.fa,b4.fa,b5.fa,b6.fa
1 2 3 4 5 6 1 2 3 4 5 6


## Generate intensity-annotated FASTA

We generate a FASTA file containing both background and spike-in entries.

In [5]:
python3 util_scripts/lfqtk_orig/lfqtk.py generate_spikein_set \
    --background_fa data/uniprot_ecoli.pure.fasta \
    --spikein_fa data/uniprot_potato.pure.fasta \
    --offset_mean 0 \
    --offset_std 0 \
    --base_int 1000000 \
    --noise_std 200000 \
    --back_count 50 \
    --spike_count 10 \
    --spike_folds "1,1,1,1,1,1,2,2,2,2,2,2" \
    --offset_folds "0,0,0,1,1,1,0,0,0,1,1,1" \
    --out_base ${run} \
    --sample_names ${file_names} \
    --verbose

4436 entries loaded from data/uniprot_ecoli.pure.fasta as background
403 entries loaded from data/uniprot_potato.pure.fasta as spike-in
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/a1.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/a2.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/a3.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/a4.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/a5.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/a6.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/b1.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/b2.fa
50 entries picked as background, 10 as spike-in
60 entries written to batch_nobatch/b3.fa
50 entries picked as background, 10 as spike-in
60 ent

# Simulate MS-data

Generate the OpenMS tool `MSSimulator` using the generated FASTA files with assigned intensities. The used output is in `featureXML`-format which contains the annotation for the entry from which it was derived as well as intensity values that could be used for downstream quantification.

In [6]:
time parallel -j ${threads} \
"echo \"Processing {}\"
MSSimulator \
    -in ${run}/{}.fa \
    -out_fm ${run}/{}.ground.featureXML" \
    ::: ${sample_names}


Processing a6
Loading sequence data from batch_nobatch/a6.fa ...
done (60 protein(s) loaded)
Starting simulation
2017/11/13, 12:26:40: Digest Simulation ... started
2017/11/13, 12:26:40: RT Simulation ... started
2017/11/13, 12:26:43: Predicting RT ... done
2017/11/13, 12:26:43: RT prediction gave 'invalid' results for 1694 peptide(s), making them unobservable.
2017/11/13, 12:26:43:   (List is too big to show)
2017/11/13, 12:26:43: Creating experiment with #501 scans ... done
2017/11/13, 12:26:43: Detectability Simulation ... started
2017/11/13, 12:26:43: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/13, 12:26:43: Simulating 1963 features
Progress of 'Ionization':
-- done [took 14.08 s (CPU), 14.19 s (Wall)] -- 
2017/11/13, 12:26:58: #Peptides not ionized: 0
2017/11/13, 12:26:58: #Peptides outside mz range: 736
2017/11/13, 12:26:58: Raw MS1 Simulation ... started
2017/11/13, 12:26:58:   Simulating signal for 5485 features ...
Progress of 'Ra

2017/11/13, 12:27:10: Contaminants out-of-RT-range: 204 / 486
2017/11/13, 12:27:10: Contaminants out-of-MZ-range: 111 / 486
2017/11/13, 12:27:12: Compressed data to grid ... 9967645 --> 8956759 (89%)
2017/11/13, 12:27:12: Adding white noise to spectra ...
2017/11/13, 12:27:12: Adding detector noise to spectra ...
2017/11/13, 12:27:12: Detector noise was disabled.
2017/11/13, 12:27:12: Tandem MS Simulation ... disabled
2017/11/13, 12:27:12: Final number of simulated features: 5484
2017/11/13, 12:27:12: Simulation took 31.754448 seconds
2017/11/13, 12:27:12: Storing simulated features in: batch_nobatch/a2.ground.featureXML
2017/11/13, 12:27:13: MSSimulator took 32.54 s (wall), 32.28 s (CPU), 0.00 s (system), 32.28 s (user).
Processing a5
Loading sequence data from batch_nobatch/a5.fa ...
done (60 protein(s) loaded)
Starting simulation
2017/11/13, 12:26:40: Digest Simulation ... started
2017/11/13, 12:26:40: RT Simulation ... started
2017/11/13, 12:26:43: Predicting RT ... done
2017/11/13

2017/11/13, 12:27:15:   (List is too big to show)
2017/11/13, 12:27:15: Creating experiment with #501 scans ... done
2017/11/13, 12:27:15: Detectability Simulation ... started
2017/11/13, 12:27:15: Ionization Simulation ... started
esi_impurity_probabilities_[0]: 1
weights[0]: 10
2017/11/13, 12:27:15: Simulating 1956 features
Progress of 'Ionization':
-- done [took 12.28 s (CPU), 12.29 s (Wall)] -- 
2017/11/13, 12:27:27: #Peptides not ionized: 0
2017/11/13, 12:27:27: #Peptides outside mz range: 719
2017/11/13, 12:27:27: Raw MS1 Simulation ... started
2017/11/13, 12:27:27:   Simulating signal for 5465 features ...
Progress of 'RawMSSignal':
-- done [took 5.82 s (CPU), 5.86 s (Wall)] -- 
2017/11/13, 12:27:34: Contaminants out-of-RT-range: 204 / 486
2017/11/13, 12:27:34: Contaminants out-of-MZ-range: 111 / 486
2017/11/13, 12:27:35: Compressed data to grid ... 10055721 --> 9011278 (89%)
2017/11/13, 12:27:35: Adding white noise to spectra ...
2017/11/13, 12:27:35: Adding detector noise to s

# Quantification

Each of the `featureXML` files are quantified using the `ProteinQuantifier` software. 

In [7]:
for xml in ${run}/*.featureXML; do 
    ProteinQuantifier \
        -in ${xml} \
        -peptide_out ${xml%.*}.csv
done


Processing summary - number of...
...features: 5485 used for quantification, 5485 total (0 no annotation, 0 ambiguous annotation)
...peptides: 1958 quantified, 1958 identified (considering best hits only)
ProteinQuantifier took 1.20 s (wall), 1.17 s (CPU), 0.00 s (system), 1.17 s (user).

Processing summary - number of...
...features: 5484 used for quantification, 5484 total (0 no annotation, 0 ambiguous annotation)
...peptides: 1958 quantified, 1958 identified (considering best hits only)
ProteinQuantifier took 1.20 s (wall), 1.15 s (CPU), 0.00 s (system), 1.15 s (user).

Processing summary - number of...
...features: 5486 used for quantification, 5486 total (0 no annotation, 0 ambiguous annotation)
...peptides: 1958 quantified, 1958 identified (considering best hits only)
ProteinQuantifier took 1.20 s (wall), 1.17 s (CPU), 0.00 s (system), 1.17 s (user).

Processing summary - number of...
...features: 5487 used for quantification, 5487 total (0 no annotation, 0 ambiguous annotation)

We receive identical peptide setup due to same protein being picked and cleaved (even if intensities varies).

## Prepare quantity matrix

The resulting quantities for the individual samples are combined into a single feature/sample-intensity matrix.

In [8]:
python3 util_scripts/combine_quant_pd.py \
    --dfs ${run}/*.csv \
    --out_fp ${run}/full_quant.tsv

In [9]:
wc -l ${run}/full_quant.tsv
head ${run}/full_quant.tsv

1944 batch_nobatch/full_quant.tsv
peptide	protein	a1	a2	a3	a4	a5	a6	b1	b2	b3	b4	b5	b6
FCR	sp|P0A7T7|RS18_ECOLI	214429092.0	140342700.0	141660496.0	154728400.0	134694504.0	190445100.0	200111300.0	147518496.0	150294600.0	142978900.0	126694296.0	111033500.0
LCR	sp|Q9JMR4|YUBK_ECOLI	60632400.0	51754800.0	27010700.0	29371800.0	45521500.0	44199300.0	40610500.0	24241700.0	27215600.0	16278100.0	10454500.0	54161500.0
FCQR	sp|P39357|YJHF_ECOLI	82801500.0	106809704.0	103172104.0	126872000.0	116927200.0	128321300.0	99170704.0	121863904.0	75656900.0	104284800.0	120258496.0	120971600.0
FLFK	sp|P34094|PHYB_SOLTU	106088600.0	72783000.0	94655496.0	109830600.0	83981800.0	88308200.0	203640192.0	335593500.0	261984792.0	274799092.0	274744600.0	212829792.0
FYLS	sp|A5A617|YDGU_ECOLI	76946000.0	82663800.0	89198496.0	61752800.0	81193504.0	64366700.0	77109296.0	96031200.0	62014200.0	71508000.0	66307400.0	67768704.0
KFCR	sp|P0A7T7|RS18_ECOLI	105538700.0	68345670.0	69978620.0	76397306.0	64302180.0	97507860.0	9633